Bibliotecas


In [446]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


Lendo dataframes


In [447]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_3348\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [469]:
userItem=dataOriginal.pivot(index='user id',columns='item id',values='rating').fillna(0)
# userItem=userItem.to_numpy()
userItem

item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [449]:
mediaUser=dataOriginal.drop(['timestamp','item id'],axis=1).groupby('user id').mean().reset_index()
# mediaUser=mediaUser.to_numpy()
mediaUser

,user id,rating
0,1,3.610294
1,2,3.709677
2,3,2.796296
3,4,4.333333
4,5,2.874286
...,...,...
938,939,4.265306
939,940,3.457944
940,941,4.045455
941,942,4.265823


In [450]:
def h(rat,user):
    val=rat - mediaUser[mediaUser['user id']==user]['rating'].values[0]
    return val

In [451]:
#iniciando
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [452]:
def cossine_similarity(u, v,tabelaSimilaridade):
    u=u.reset_index()
    v=v.reset_index()
    itemsComuns = set(u['item id']).intersection(set(v['item id']))  
    id_u=u['user id'][0]
    id_v = v['user id'][0]
    u = u[u['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    v = v[v['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    if((np.linalg.norm(u)*np.linalg.norm(v))==0):
        return 0
    result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
    tabelaSimilaridade[id_u][id_v]=result
    tabelaSimilaridade[id_v][id_u]=result

    return result



In [453]:
def cossine_similarity2(u, v,tabelaSimilaridade):
    u=u.reset_index()
    v=v.reset_index()
    itemsComuns = set(u['item id']).intersection(set(v['item id']))  
    id_u=u['user id'][0]
    id_v = v['user id'][0]
    u = u[u['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    v = v[v['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    if(u.empty or v.empty):
        return 0
    result = cosine_similarity([u],[v])[0][0]
    tabelaSimilaridade[id_u][id_v]=result
    tabelaSimilaridade[id_v][id_u]=result

    return result

In [456]:

def user_based(user_id, item_id,data,tabelaSimilaridade,tabelaCompleta=False):
    if ~tabelaCompleta:
        Nui = userData[userData['user id'].isin(
            data[data['item id'] == item_id]['user id'])]['user id']
    
        if(len(Nui)==0):
            return None

        parcial1=sum(Nui.apply(lambda x:
                               ((cossine_similarity2(data[data['user id'] == user_id],
                                                    data[data['user id']
                                                        == x],tabelaSimilaridade
                                                    )if math.isnan(tabelaSimilaridade[user_id][x])else tabelaSimilaridade[user_id][x])
                                                    *h(data[(
                                                        data['user id'] == x) & (data['item id'] == item_id)
                                ]['rating'],x).reset_index(drop=True)
                                )).rename(columns={0: 'rating'})['rating'])
        
        parcial2 = sum(Nui.apply(lambda x:
                                (abs(cossine_similarity2(data[data['user id'] == user_id],
                                                        data[data['user id']
                                                            == x],
                                                            tabelaSimilaridade
                                                        )) if math.isnan( tabelaSimilaridade[user_id][x])  else abs(tabelaSimilaridade[user_id][x]) )
                                ))
        resp=(parcial1/parcial2)+mediaUser[mediaUser['user id']==user_id]['rating'].values[0]
        return resp
    

In [457]:
test=dataOriginal.drop(columns='timestamp')
train,test1=train_test_split(test,train_size=0.2)
x_test=train.drop('rating',axis=1)
y_test=train['rating']
x_test

,user id,item id
27670,286,737
45763,234,510
6820,56,98
77578,805,866
96807,268,116
...,...,...
72913,902,176
38794,595,291
80996,788,44
13568,139,286


In [458]:
y_test_predict= x_test.apply(lambda x:
           user_based(x['user id'],x['item id'],test,similaridade) ,axis=1)
y_test_predict

27670    3.177856
45763    3.465178
6820     4.301621
77578    3.094322
96807    3.170071
           ...   
72913    3.799388
38794    3.079494
80996    3.126917
13568    4.101967
62582    2.817164
Length: 20000, dtype: float64

In [459]:
y_predict=y_test_predict.replace(np.nan,0)
y_predict[y_predict==0]

Series([], dtype: float64)

In [460]:
y_test

27670    4
45763    4
6820     4
77578    1
96807    4
        ..
72913    5
38794    3
80996    4
13568    4
62582    3
Name: rating, Length: 20000, dtype: int64

In [461]:
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,0.960582,0.857075,0.919264,0.932614,0.953845,0.943945,0.977505,0.976404,0.968304,...,0.942682,0.875338,0.965889,0.91633,0.910709,0.944993,0.947752,0.970836,0.923505,0.929744
2,0.960582,1,0.935601,0.946756,0.984803,0.955547,0.967087,0.958659,0.940056,0.982614,...,0.950765,0.957073,0.9674,0.966063,0.946455,0.974901,0.93611,NaN,0.948493,0.977474
3,0.857075,0.935601,1,0.919528,NaN,0.890713,0.879543,0.89589,0.919145,0.902444,...,NaN,NaN,0.896328,0.840511,0.872572,0.993884,0.852826,NaN,0.912009,0.989949
4,0.919264,0.946756,0.919528,1,0.994692,0.936996,0.882332,0.976533,0.993884,0.969117,...,0.982905,NaN,0.952164,0.895669,0.958333,1.0,0.96926,0.998538,0.980173,0.976417
5,0.932614,0.984803,NaN,0.994692,1,0.933627,0.908163,0.941228,0.880919,0.940114,...,0.920066,0.943281,0.941549,0.965824,0.903752,0.943701,0.918769,0.975643,0.946235,0.909228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.944993,0.974901,0.993884,1.0,0.943701,0.928228,0.957014,0.989011,1.0,0.993822,...,0.951155,0.962738,0.975279,0.921936,0.944249,1,0.973308,0.965729,0.980407,0.954349
940,0.947752,0.93611,0.852826,0.96926,0.918769,0.944746,0.941044,0.95436,0.982175,0.967073,...,0.936986,0.95277,0.918801,0.880023,0.926234,0.973308,1,0.967752,0.960041,0.962575
941,0.970836,NaN,NaN,0.998538,0.975643,0.945658,0.9374,0.931134,1.0,NaN,...,0.943701,0.962572,0.974237,0.952969,0.934363,0.965729,0.967752,1,0.982302,0.960754


In [462]:
mean_absolute_error(y_test,y_predict)

0.7335151257160113

In [466]:
mean_squared_error(y_test,y_predict)

0.8627301156180099